In [ ]:
import os, sys

# Clone repo
if not os.path.exists('/content/qwen3_tts'):
    os.system('git clone https://github.com/QwenLM/Qwen3-TTS /content/qwen3_tts')
else:
    print("✅ Repo già presente")

# Dipendenze esatte da pyproject.toml
os.system('pip install -q transformers==4.57.3 accelerate==1.12.0')
os.system('pip install -q soundfile librosa torchaudio onnxruntime einops sox')

# Installa il pacchetto qwen_tts (senza -e)
os.system('pip install -q /content/qwen3_tts')

# Dipendenze per le funzioni extra
os.system('pip install -q gradio pydub pyloudnorm noisereduce')

print("✅ Tutto installato. Riavvio...")
os.kill(os.getpid(), 9)


In [ ]:
import sys, torch

# Doppia sicurezza: sia pip che path diretto
sys.path.insert(0, '/content/qwen3_tts')

from qwen_tts import Qwen3TTSModel

print(f"✅ Python: {sys.version[:6]}")
print(f"✅ Qwen3TTSModel importato")
print(f"🎮 CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    total = torch.cuda.get_device_properties(0).total_memory / 1024**3
    free  = torch.cuda.mem_get_info()[0] / 1024**3
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {free:.1f}GB liberi / {total:.1f}GB totali")
else:
    print("⚠️ Nessuna GPU → vai su Runtime → Change runtime type → T4 GPU")



    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    



********
********
 
✅ Python: 3.12.1
✅ Qwen3TTSModel importato
🎮 CUDA: True
   GPU: Tesla T4
   VRAM: 14.5GB liberi / 14.6GB totali


In [ ]:
print("📦 Caricamento Qwen3-TTS 1.7B Base (3-5 min)...")

model = Qwen3TTSModel.from_pretrained(
    "Qwen/Qwen3-TTS-12Hz-1.7B-Base",
    device_map="cuda:0",
    dtype=torch.float16,
    attn_implementation="sdpa",
)

SAMPLE_RATE = 24000
print("✅ Modello pronto!")


📦 Caricamento Qwen3-TTS 1.7B Base (3-5 min)...


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Modello pronto!


In [ ]:
import os, time, hashlib, tempfile, re, subprocess, sys, importlib.util
import gradio as gr
import torch, numpy as np
import librosa, soundfile as sf
from pydub import AudioSegment
from pydub.silence import detect_leading_silence
import pyloudnorm as pyln
import noisereduce as nr

# ─────────────────────────────────────────────
# (1) Whisper per auto-trascrizione reference
# ─────────────────────────────────────────────
if importlib.util.find_spec("faster_whisper") is None:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "faster-whisper"], check=True)

from faster_whisper import WhisperModel

_whisper = None
def get_whisper(model_size="base"):
    global _whisper
    if _whisper is None or getattr(_whisper, "_size", None) != model_size:
        # base/small sono un buon compromesso su Colab T4
        _whisper = WhisperModel(model_size, device="cuda" if torch.cuda.is_available() else "cpu",
                                compute_type="float16" if torch.cuda.is_available() else "int8")
        _whisper._size = model_size
    return _whisper

def trascrivi_reference(audio_path, lang="it", whisper_size="base"):
    wm = get_whisper(whisper_size)
    segments, info = wm.transcribe(audio_path, language=lang, vad_filter=True)
    text = " ".join([s.text.strip() for s in segments]).strip()
    return text

# ─────────────────────────────────────────────
# (2) Preprocessing testo IT (evita “punto”, virgolette, ecc.)
# ─────────────────────────────────────────────
def preprocessa_testo_it(testo: str):
    testo = testo.replace('«','').replace('»','')
    testo = testo.replace('"','')
    testo = testo.replace('\u2018',"'").replace('\u2019',"'")  # apostrofi tipografici
    # '.' a fine frase -> ',' (pausa senza “punto”)
    testo = re.sub(r'\.(\s+|$)', r',\1', testo)
    # puntini -> virgola
    testo = testo.replace('…', ',')
    testo = re.sub(r'\.{2,}', ',', testo)
    # trattini lunghi -> virgola
    testo = testo.replace('—', ',').replace('–', ',')
    # parentesi via
    testo = re.sub(r'[()[\]{}]', '', testo)
    # spazi multipli
    testo = re.sub(r'\s+', ' ', testo).strip()
    return testo

# ─────────────────────────────────────────────
# (3) Reference audio: MP3/WAV lungo -> chunk “migliore” 15–20s
# ─────────────────────────────────────────────
def trim_silence(audio, threshold=-40, padding_ms=120):
    start = detect_leading_silence(audio, silence_threshold=threshold)
    end   = detect_leading_silence(audio.reverse(), silence_threshold=threshold)
    trimmed = audio[start : len(audio) - end]
    pad = AudioSegment.silent(duration=padding_ms)
    return pad + trimmed + pad

def prepara_reference(filepath, denoise=False, target_ms=18_000):
    audio = AudioSegment.from_file(filepath)
    audio = audio.set_channels(1).set_frame_rate(22050)
    audio = trim_silence(audio)

    # Se lungo: estrai finestra più parlata (RMS max)
    if len(audio) > target_ms:
        step = 1500
        chunks = [audio[i:i+target_ms] for i in range(0, len(audio) - target_ms, step)]
        audio = max(chunks, key=lambda c: c.rms)

    tmp = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    audio.export(tmp.name, format="wav")

    if denoise:
        data, sr = sf.read(tmp.name)
        data = nr.reduce_noise(y=data, sr=sr, prop_decrease=0.75)
        sf.write(tmp.name, data, sr)

    return tmp.name

# ─────────────────────────────────────────────
# (4) Post-processing output: trim + LUFS
# ─────────────────────────────────────────────
def postprocessa_output(wav_path, target_lufs=-18.0):
    audio = AudioSegment.from_wav(wav_path)
    audio = trim_silence(audio, threshold=-50, padding_ms=150)
    audio.export(wav_path, format="wav")

    data, rate = sf.read(wav_path)
    meter = pyln.Meter(rate)
    loudness = meter.integrated_loudness(data)
    if loudness > -70:
        data = pyln.normalize.loudness(data, loudness, target_lufs)
        sf.write(wav_path, data, rate)
    return wav_path

# ─────────────────────────────────────────────
# (5) Target durata (pitch-preserving time-stretch)
# ─────────────────────────────────────────────
def durata_wav(path):
    data, rate = sf.read(path)
    return len(data) / rate

def adatta_durata(wav_path, target_sec):
    data, sr = sf.read(wav_path)
    current_sec = len(data) / sr
    rate = current_sec / target_sec
    rate = max(0.7, min(1.4, rate))  # range “sicuro”
    stretched = librosa.effects.time_stretch(data.astype(np.float32), rate=rate)
    sf.write(wav_path, stretched, sr)
    return wav_path, rate, current_sec

# ─────────────────────────────────────────────
# (6) Cache voice_clone_prompt (velocizza ripetizioni)
# ─────────────────────────────────────────────
def _hash_file(path):
    with open(path, "rb") as f:
        return hashlib.md5(f.read()).hexdigest()

_prompt_cache = {}
def get_voice_prompt(ref_wav, ref_text, xvec_only):
    key = (_hash_file(ref_wav), ref_text, bool(xvec_only))
    if key not in _prompt_cache:
        _prompt_cache[key] = model.create_voice_clone_prompt(
            ref_audio=ref_wav,
            ref_text=ref_text,
            x_vector_only_mode=bool(xvec_only),
        )
    return _prompt_cache[key]

# ─────────────────────────────────────────────
# (7) Preset “Italian-friendly” (valori conservativi)
# ─────────────────────────────────────────────
PRESET = {
    "🇮🇹 IT – Naturale YouTube (consigliato)": dict(
        temperature=0.85, top_p=0.95, top_k=50, repetition_penalty=1.06,
        max_new_tokens=3072
    ),
    "🇮🇹 IT – Super stabile (meno errori)": dict(
        temperature=0.70, top_p=0.85, top_k=40, repetition_penalty=1.10,
        max_new_tokens=3072
    ),
    "🇮🇹 IT – Più espressivo (rischio errori)": dict(
        temperature=1.00, top_p=0.95, top_k=70, repetition_penalty=1.05,
        max_new_tokens=3072
    ),
}

def applica_preset(nome):
    p = PRESET[nome]
    return p["temperature"], p["top_p"], p["top_k"], p["repetition_penalty"], p["max_new_tokens"]

# ─────────────────────────────────────────────
# (8) Generazione Qwen Voice Clone
# ─────────────────────────────────────────────
def genera_qwen_it(
    testo, ref_file,
    ref_text_manual,
    auto_ref_text, whisper_size,
    xvec_only, use_cache,
    denoise_ref, do_postprocess,
    preset_name,
    temperature, top_p, top_k, repetition_penalty, max_new_tokens,
    target_min, target_sec,
):
    if not testo or not testo.strip():
        raise gr.Error("Inserisci un testo.")
    if ref_file is None:
        raise gr.Error("Carica un audio di riferimento.")

    t0 = time.time()
    target_totale = float(target_min) * 60 + float(target_sec)

    testo_proc = preprocessa_testo_it(testo)

    # prepara reference (estrae chunk migliore se lungo)
    ref_wav = prepara_reference(ref_file, denoise=denoise_ref, target_ms=18_000)

    # ref_text: manuale > auto(whisper) > vuoto
    ref_text = (ref_text_manual or "").strip()
    if (not xvec_only) and (not ref_text) and auto_ref_text:
        ref_text = trascrivi_reference(ref_wav, lang="it", whisper_size=whisper_size)

    # Avviso utile: se xvec_only=False e ref_text vuoto, la qualità può peggiorare
    warn = ""
    if (not xvec_only) and (not ref_text):
        warn = "⚠️ ref_text vuoto con x_vector_only_mode=False: meglio inserire o auto-trascrivere."

    gen_kwargs = dict(
        max_new_tokens=int(max_new_tokens),
        do_sample=True,
        top_k=int(top_k),
        top_p=float(top_p),
        temperature=float(temperature),
        repetition_penalty=float(repetition_penalty),
        subtalker_dosample=True,
        subtalker_top_k=int(top_k),
        subtalker_top_p=float(top_p),
        subtalker_temperature=float(temperature),
    )

    if use_cache:
        prompt = get_voice_prompt(ref_wav, ref_text, xvec_only)
        wavs, sr = model.generate_voice_clone(
            text=testo_proc,
            language="Italian",
            voice_clone_prompt=prompt,
            **gen_kwargs,
        )
    else:
        wavs, sr = model.generate_voice_clone(
            text=testo_proc,
            language="Italian",
            ref_audio=ref_wav,
            ref_text=ref_text,
            x_vector_only_mode=bool(xvec_only),
            **gen_kwargs,
        )

    out_path = "/content/output_qwen3_it.wav"
    sf.write(out_path, wavs[0], sr)

    # target durata
    stretch = 1.0
    if target_totale > 0:
        out_path, stretch, _ = adatta_durata(out_path, target_totale)

    if do_postprocess:
        postprocessa_output(out_path, target_lufs=-18.0)

    dur = durata_wav(out_path)
    info = f"✅ {time.time()-t0:.1f}s | durata {dur:.1f}s | preset: {preset_name} | stretch {stretch:.3f}x"
    if target_totale > 0:
        info += f" | delta {dur-target_totale:+.1f}s"
    if warn:
        info = warn + "\n" + info

    return out_path, testo_proc, ref_text, info

# ─────────────────────────────────────────────
# (9) Gradio UI
# ─────────────────────────────────────────────
with gr.Blocks(title="Qwen3-TTS — Voice Clone IT Pro") as ui:
    gr.Markdown("# Qwen3‑TTS — Voice Cloning Italiano (Pro)")
    gr.Markdown("Nota: lo **stile vocale** non è un parametro della modalità Voice Clone; la qualità in Voice Clone migliora soprattutto con ref_text (qui puoi auto‑trascriverlo).")

    with gr.Row():
        with gr.Column(scale=1):
            ref_audio = gr.Audio(label="🎤 Reference (WAV/MP3, anche lungo)", type="filepath", sources=["upload"])

            ref_text_manual = gr.Textbox(
                label="📝 Trascrizione reference (migliora fedeltà)",
                placeholder="Se la inserisci è meglio. In alternativa attiva auto‑trascrizione.",
                lines=2
            )

            auto_ref_text = gr.Checkbox(label="🧠 Auto‑trascrivi reference (Whisper)", value=True)
            whisper_size = gr.Dropdown(label="Whisper model", choices=["tiny","base","small"], value="base")

            xvec_only = gr.Checkbox(
                label="⚡ x_vector_only_mode (non richiede trascrizione, qualità inferiore)",
                value=False
            )
            use_cache = gr.Checkbox(label="⚡ Cache voice_clone_prompt (consigliato)", value=True)

            with gr.Row():
                denoise_ref = gr.Checkbox(label="🔇 Denoise reference", value=False)
                do_postprocess = gr.Checkbox(label="🎚️ Normalizza output (-18 LUFS)", value=True)

            preset_dd = gr.Dropdown(
                label="🎛️ Preset IT",
                choices=list(PRESET.keys()),
                value="🇮🇹 IT – Naturale YouTube (consigliato)"
            )

            gr.Markdown("### 🎬 Target durata (opzionale)")
            with gr.Row():
                target_min = gr.Number(label="Min", value=0, minimum=0, maximum=60, step=1)
                target_sec = gr.Number(label="Sec", value=0, minimum=0, maximum=59, step=1)

            with gr.Accordion("⚙️ Avanzate", open=False):
                temperature = gr.Slider(label="temperature", minimum=0.3, maximum=1.3, value=0.85, step=0.05)
                top_p = gr.Slider(label="top_p", minimum=0.5, maximum=1.0, value=0.95, step=0.05)
                top_k = gr.Slider(label="top_k", minimum=1, maximum=200, value=50, step=1)
                repetition_penalty = gr.Slider(label="repetition_penalty", minimum=1.0, maximum=1.3, value=1.06, step=0.01)
                max_new_tokens = gr.Slider(label="max_new_tokens", minimum=512, maximum=8192, value=3072, step=256)

            preset_dd.change(
                fn=applica_preset,
                inputs=preset_dd,
                outputs=[temperature, top_p, top_k, repetition_penalty, max_new_tokens]
            )

        with gr.Column(scale=1):
            testo_input = gr.Textbox(label="📄 Testo (Italiano)", lines=12, placeholder="Incolla qui il tuo voiceover in italiano…")
            btn = gr.Button("🚀 Genera", variant="primary")
            audio_out = gr.Audio(label="🔊 Output", type="filepath")
            testo_proc_out = gr.Textbox(label="Testo preprocessato", lines=3)
            ref_text_out = gr.Textbox(label="ref_text usato (manuale o auto)", lines=3)
            info_out = gr.Textbox(label="Info", lines=3)

    btn.click(
        fn=genera_qwen_it,
        inputs=[
            testo_input, ref_audio,
            ref_text_manual,
            auto_ref_text, whisper_size,
            xvec_only, use_cache,
            denoise_ref, do_postprocess,
            preset_dd,
            temperature, top_p, top_k, repetition_penalty, max_new_tokens,
            target_min, target_sec,
        ],
        outputs=[audio_out, testo_proc_out, ref_text_out, info_out]
    )

ui.queue(default_concurrency_limit=1)
ui.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b5786121738a56b93a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
